
    This Source Code Form is subject to the terms of the Mozilla Public
    License, v. 2.0. If a copy of the MPL was not distributed with this
    file, You can obtain one at http://mozilla.org/MPL/2.0/.


In [ ]:
# Enable automatic module reload
%load_ext autoreload
%autoreload 2

# To ease the loading of modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Load BEACS module
#from beacs.agents.beacs import BEACS, Configuration

# Load particular Environment Adapter
#from example.adapter.CartPoleEnvironmentAdapter import CartPoleEnvironmentAdapter

# Load Metrics
#from example.metrics.CartPoleMetrics import _cartpole_metrics

# Load Plotting Wrappers
#from example.metrics.CartPolePlottingWrapper import \
#    parse_metrics_to_df, \
#    plot_cartpole_performance, \
#    plot_average_cartpole_performance

# Load environments
import gym, random

# Agent - BEACS

## Common parameters

In [ ]:
#Environmental Set Up
CLASSIFIER_LENGTH = 4
NUMBER_OF_POSSIBLE_ACTIONS = 2
CARRACING = "CarRacing-v0"

#Exploration Set Up
NUMBER_OF_EXPLORE_TRIALS = 500
METRICS_TRIAL_FREQUENCY_EXPLORE = 1
EPSILON = 0.5

#Exploitation Set Up
NUMBER_OF_EXPLOIT_TRIALS_NO_RL = 100
BETA_EXPLOIT_NO_RL = 0.00
NUMBER_OF_EXPLOIT_TRIALS_RL = 500
BETA_EXPLOIT_RL = 0.05

#RL Set Up
GAMMA = 0.95
BETA_RL = 0.05

#BEACS Set Up
DO_GA = False
ENABLE_PEP = True
LENGTH_OF_BEHAVIORAL_SEQUENCES = 0
BETA_ALP = 0.05

## Environment - CarRacing

In [ ]:
# Initialize environment
CarRacing = gym.make(CARRACING)

# Reset it, by putting an agent into random position
CarRacing.reset()

ACTION = {
    'Turn_left':[ -1.0, 0.0, 0.0 ],
    'Turn_right':[ +1.0, 0.0, 0.0 ],
    'Accelerate':[ 0.0, 1.0, 0.0 ],
    'Brake':[ 0.0, 0.0, 0.8 ],
    'Do-Nothing':[ 0.0, 0.0, 0.0 ],
}

for _ in range(50):
    CarRacing.render()
    observation, reward, done, info = CarRacing.step(ACTION['Do-Nothing'])
    if done:
        break

import numpy as np
import matplotlib.pyplot as plt
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

for plop in range(200):
    CarRacing.render()
    observation, reward, done, info = CarRacing.step(ACTION['Accelerate'])
    #observation, reward, done, info = CarRacing.step(ACTION[random.choice(list(ACTION.keys()))])
    observation = observation[:84, :] # 84 x 96 (PGCD = 12 donc 7 x 8)
    gray = rgb2gray(observation)
    diviseur = 12
    new_width = 84 // diviseur
    new_length = 96 // diviseur
    new_gray = np.arange(new_width*new_length).reshape(new_width,new_length)
    for i in range(new_width):
        for j in range(new_length):
            mean = 0.
            for k in range(diviseur):
                for l in range(diviseur):
                    mean += gray[i*diviseur+k, j*diviseur+l]
            mean = mean / (diviseur*diviseur)
            new_gray[i][j] = mean
    mean = 0.
    for i in range(new_width):
        for j in range(new_length):
            mean += new_gray[i][j]
    mean = mean / (new_width*new_length)
    for i in range(new_width):
        for j in range(new_length):
            #if new_gray[i][j] <= mean:
            if new_gray[i][j] <= 130:
                new_gray[i][j] = 255
            else:
                new_gray[i][j] = 0
    #if plop % 20 == 0:
    #    plt.imshow(observation)
    #    plt.show()
    #    plt.imshow(new_gray, cmap=plt.get_cmap('gray'))
    #    plt.show()
    if done:
        break
CarRacing.close()